[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/Previsão/Prev_Demanda.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**


Código produzido a partir de: https://medium.com/mlearning-ai/a-stochastic-model-for-demand-forecating-in-python-a1b568b80b94

# Previsão de Demanda - Médias Móveis

Nessa aula, vamos fazer algumas análises para previsão de demanda no Python. Para isso, utilizaremos alguns conjuntos de dados disponíveis no Kaggle.
 

### Importando Bibliotecas

Importaremos as seguintes bibliotecas:

- pandas: biblioteca muito utilizada para análise de dados em formato tabular (assim como um Excel).
- matplotlib.pyplot: pacote vastamente utilizado para visualizações.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

### Importação e limpeza dos dados

Inicialmente, vamos importar os dados.

In [ ]:
# Como os nossos dados possuem uma coluna de datas, avisaremos isso ao python. 

url = 'https://raw.githubusercontent.com/diogoflim/MGP/main/GP/dados/dados_passageiros.csv'


dados_brutos = pd.read_csv(url, index_col=0, parse_dates=["Month"])
dados_brutos

Nossos dados parecem estar organizados. Vou apenas renomear as colunas. 

Antes disso, criarei uma cópia para deixarmos guardados os dados brutos 

In [ ]:
dados_modificados = dados_brutos.rename(columns={"#Passengers": "Passageiros"}).rename_axis("Data") # Muda o nome das colunas
dados = dados_modificados.copy()
dados

### Visualização inicial

Inicialmente, podemos perceber que nossos temos 144 observações de demandas mensais de passageiros, entre 1949 e 1960.

Uma visualização básica pode nos ajudar a entender o funcionamento da demanda.

Vamos usar um simples gráfico de linha.

In [ ]:
plt.plot(dados)
plt.show()

A demanda cresceu ao longo dos anos. Porém, um comportamento cíclico parece ter sido detectado.

O que poderia ser?

- **Talvez alguns meses (exemplo: férias) impactem a demanda por passageiros.**  

Veremos neste notebook como considerar o efeito sazonal. Inicialmente, veja como calcular a média por mês:

In [ ]:
dados.groupby(by=[dados.index.month]).mean().rename_axis("Mês")

A análise indica uma maior média nos meses de **verão no hemisfério norte!**

Vamos seguir para os modelos de séries temporais.

### Média móvel simples (SMA)


Vamos aplicar uma média móvel simples com tempo igual a 3

In [ ]:
k=3
dados.rolling(k).mean().shift(1)

### Criando uma nova coluna com os resultados

Criaremos uma nova coluna que receberá essa análise!


In [ ]:
dados["MM_3"]= dados["Passageiros"].rolling(k).mean().shift(1)
dados

### Variando o malor do hiperparâmetro

Seguindo esse raciocínio, poderíamos fazer várias ao mesmo tempo

In [ ]:
dados_sma = dados_modificados.copy()

for k in [3, 6, 9, 12]: 
    dados_sma ["MM_" + str(k)] = dados_sma["Passageiros"].rolling(k).mean().shift(1)

dados_sma

In [ ]:
dados_sma.plot(figsize=(16,12))

### Interpretando a escolha de $k$

Perceba que valores mais altos de $k$ de alguma maneira suavizam a média! Estamos perdendo o efeito sazonal.

Vamos analisar o modelo de suavização exponencial.


### Média Móvel Exponencialmente Ponderada (EWMA)

In [ ]:
dados_ewma = dados_modificados.copy()

In [ ]:
dados_ewma["alfa=0,5"] = dados_ewma["Passageiros"].ewm(alpha=0.5, adjust=False).mean()
dados_ewma

### Considerando a Sazonalidade

Para considerar a sazonalidade, usaremos o procedimento visto na aula:

O efeito sazonal é dado por: $$\frac{\text{𝑚é𝑑𝑖𝑎 𝑑𝑜 𝑚ê𝑠}}{\text{𝑚é𝑑𝑖𝑎 𝑡𝑜𝑡𝑎𝑙}}$$

Uma vez calculado, faremos os passos:

1. Use a fórmula a seguir para ajustar os dados com o efeito sazonal: $ 𝑉𝑎𝑙𝑜𝑟 𝑎𝑗𝑢𝑠𝑡𝑎𝑑𝑜 = \frac{\text{𝑉𝑎𝑙𝑜𝑟 𝑜𝑏𝑠𝑒𝑟𝑣𝑎𝑑𝑜}}{\text{𝐸𝑓𝑒𝑖𝑡𝑜 𝑆𝑎𝑧𝑜𝑛𝑎𝑙}}$

2. Escolha um método de previsão

3. Aplique o método de previsão aos dados ajustados (que consideram o efeito sazonal)

4. Multiplique essa previsão pelo efeito sazonal correspondente para obter o valor de previsão real (sem o ajuste sazonal)


In [ ]:
dados_ajustados = dados_modificados.copy()
dados_ajustados

### Calculando o Efeito Sazonal

Inicialmente, precisamos calcular o efeito sazonal. Para isso, usamos a média mensal e a média total.

In [ ]:
media_total = dados_ajustados.mean()
media_total

In [ ]:
media_mensal = dados_ajustados.groupby(by=[dados_ajustados.index.month]).mean().rename_axis("Mês")

media_mensal

In [ ]:
efeito_sazonal = media_mensal / media_total
efeito_sazonal

De posse dessa informação, vamos para o passo 1 do procedimento. Devemos dividir cada valor pelo seu efeito sazonal correspondente.

### Tabela Modificada
Vejamos a tabela com as vendas originais e as modificadas pelo efeito sazonal

In [ ]:
dados_ajustados

In [ ]:
dados_ajustados["Mês"] = dados_ajustados.index.month
dados_ajustados

In [ ]:
efeito_sazonal.loc[1]

In [ ]:
dados_ajustados["Efeito Sazonal"] = [efeito_sazonal.loc[i]["Passageiros"] for i in dados_ajustados["Mês"]]

dados_ajustados

In [ ]:
dados_ajustados["x_t ajustado"] = dados_ajustados["Passageiros"] * dados_ajustados["Efeito Sazonal"]


In [ ]:
dados_ajustados

## Previsões nos dados ajustados

Agora, podemos aplicar uma previsão usando o x_t ajustado.

Vamos utilizar a suavização exponencial.

In [ ]:
dados_ajustados["ewma_ajustado"]= dados_ajustados["x_t ajustado"].ewm(alpha=0.5, adjust=False).mean()

for k in [3, 6, 9, 12]: 
    dados_ajustados ["MM_" + str(k) + "_ajustado"] = dados_ajustados["Passageiros"].rolling(k).mean().shift(1)

dados_ajustados

## Multiplicando as previsões pelo efeito sazonal

In [ ]:
dados_ajustados["ewma"] = dados_ajustados["ewma_ajustado"] * dados_ajustados["Efeito Sazonal"]

In [ ]:
for k in [3, 6, 9, 12]:  
    dados_ajustados ["MM_" + str(k)] = dados_ajustados ["MM_" + str(k) + "_ajustado"] * dados_ajustados["Efeito Sazonal"]

In [ ]:
dados_ajustados

# Performance do Preditor

Há diversas formas de analizar a qualidade de um preditor. 

Em uma tarefa de regressão, tal como a previsão de demanda, uma medida de performance muito utilizada é o erro médio quadrático.

$$J=\frac{1}{n}\sum_{t=1}^{n} (X_t - F_t)^2$$


Esse método está disponível no Python em uma biblioteca clássica de Aprendizado de Máquina. 


Vamos utilizar essa métrica em nossa análise.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print("Erro Médio Quadrático SEM CONSIDERAR a sazonalidade:")
print(f'Suavização Exponencial com alfa = 0,5: {mean_squared_error(dados_ewma["Passageiros"].iloc[-30:], dados_ewma["alfa=0,5"].iloc[-30:])}')
print(f'Média móvel com k = 3: {mean_squared_error(dados_sma["Passageiros"].iloc[-30:], dados_sma["MM_3"].iloc[-30:])}')
print(f'Média móvel com k = 6: {mean_squared_error(dados_sma["Passageiros"].iloc[-30:], dados_sma["MM_6"].iloc[-30:])}')
print(f'Média móvel com k = 9: {mean_squared_error(dados_sma["Passageiros"].iloc[-30:], dados_sma["MM_9"].iloc[-30:])}')
print(f'Média móvel com k = 12: {mean_squared_error(dados_sma["Passageiros"].iloc[-30:], dados_sma["MM_12"].iloc[-30:])}')




print("-------------------------------------------")

print("Erro Médio Quadrático CONSIDERANDO a sazonalidade:")
print(f'Suavização Exponencial com alfa = 0,5: {mean_squared_error(dados_ajustados["Passageiros"].iloc[-30:], dados_ajustados["ewma"].iloc[-30:])}')
print(f'Média móvel com k = 3: {mean_squared_error(dados_ajustados["Passageiros"].iloc[-30:], dados_ajustados["MM_3"].iloc[-30:])}')
print(f'Média móvel com k = 6: {mean_squared_error(dados_ajustados["Passageiros"].iloc[-30:], dados_ajustados["MM_6"].iloc[-30:])}')
print(f'Média móvel com k = 9: {mean_squared_error(dados_ajustados["Passageiros"].iloc[-30:], dados_ajustados["MM_9"].iloc[-30:])}')
print(f'Média móvel com k = 12: {mean_squared_error(dados_ajustados["Passageiros"].iloc[-30:], dados_ajustados["MM_12"].iloc[-30:])}')


# Exercício

Resolva o exercício apresentado no último slide da aula de séries temporais.

In [ ]:
from datetime import datetime

In [ ]:
Vendas = [112,146,122,125,127,157,150,235,60,92,206,312,146,113,92,160,188,190,168,235,122,97,186,354,199,175,88,112,149,140,154,275,90,120,226,360]

In [ ]:
start_dt = datetime.strptime("2001-01", "%Y-%m")
end_dt = datetime.strptime("2004-01", "%Y-%m")
date_list = pd.date_range(start_dt, end_dt, freq='M')
date_list


In [ ]:
dados = pd.DataFrame({"Vendas": Vendas})
dados.index = date_list
dados